# Farm monitoring: multi-year crop extent, rotation and dynamics

## Background

Crops are important in national and international economics, trade, and food security and are a major topic of interest in the domains of policy, economics, land management, and conservation. Monitoring agricultural practices is also essential as demand for food has placed huge pressures on landscapes and particularly natural ecosystems, with these impacting (often adversely) on soils, air, water, and biodiversity. By knowing and understanding the distributions, types, and management regimes (e.g., rotational cycles) of crops, changes in management practices can be better implemented to reduce pollution, conserve and/or restore biodiversity, and control the spread of crop diseases.

              Dynamics of crop fields in Monmouthshire between Nov 2017 and Nov 2021 (Planet imagery)
<video controls autoplay width="960" height="540" src="https://storage.googleapis.com/planet-t2/monmouthshire_crops-IkhQm8S4g/movie.mp4" />


## Description

This notebook demonstrates how crop extent, rotation and dynamics can be quickly mapped/monitored over multiple years, using Sentinel-1 Synthetic Aperture Radar (SAR) satellite sensor, within the Wales Open Data Cube (`WDC`). 

By comparison with optical sensors, that are mainly sensitive to colour and chemistry, SAR data are stronly correlated to height and texture (i.e., the structure of crops). Moreover, SAR data have the advantage of operating at wavelengths not impeded by cloud cover, illumination or weather conditions, with this allowing monitoring of field plots over the course of the crop cycle including during winter period.

This notebook uses the Sentinel-1 Analyis Ready Data (ARD), as well as custom python libraries for crop monitoring using algorithms developed by and provided through the Living Wales project.

Topics include: 

1. Import an area of interest (e.g., a farm's field plots) from shapefile
2. Reporting on the annual  crop type area
3. Mapping crop rotation at farm scale
4. Mapping crop rotation at plot scale

### Jupyter Notebooks
#### Running (executing) a cell
Jupyter Notebooks allow code to be separated into sections that can be executed independent of one another.
These sections are called "cells".

The python code is written into individual cells that can be executed by placing the cursor in the cell and typing `Shift-Enter` on the keyboard or selecting the &#9658; "Run" button in the ribbon at the top of the notebook.
These options will run a single cell at a time.

When a cell is run, the cell's content is executed.
Any output produced from running the cell will appear directly below it.

Run the cell below as a test:

In [ ]:
%load_ext jupyter_black

In [ ]:
print("I ran a cell!")

#### Cell status
The `[ ]:` symbol to the left of each Code cell describes the state of the cell:

* `[ ]:` means that the cell has not been run yet.
* `[*]:` means that the cell is currently running.
* `[1]:` means that the cell has finished running and was the first cell run. The number indicates the order that the cells were run in.

***

## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell.

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

In [ ]:
# Import modules

import sys

import time
from time import time as now
import datetime as dt

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

import geopandas as gpd
from ipyleaflet import GeoData

import warnings

warnings.filterwarnings("ignore")

import datacube
from datacube.utils.geometry import Geometry, CRS

sys.path.append("../wales_utils/data_cube_utilities")
from display_tools import map_geom, draw_select
from wdc_datahandling import geom_fromdrawn, geopolygon_masking

sys.path.append("../wales_utils/themes_utilities")
from crop import crop_type_widget, report_crop_type_area, play_crop_rotation

### Connect to the datacube
Connect to the datacube so we can access Living Wales Analysis Ready Data.

In [ ]:
dc = datacube.Datacube()

### Import an area of interest (e.g., a farm's field plots) from shapefile

In [ ]:
# Loading field plots from shapefile
field_plots = gpd.read_file("../vectors/crops_study_area_27700.shp")

# Transform shapefile boundaries into geographic data (and affect a style)
geo_data = GeoData(
    geo_dataframe=field_plots.to_crs(epsg=4326),
    style={
        "color": "black",
        "fillColor": "#3366cc",
        "opacity": 0.05,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0.6,
    },
    hover_style={"fillColor": "red", "fillOpacity": 0.2},
    name="Fields",
)

# map the geographic data
m = map_geom(geo_data)
m

## Reporting on the annual  crop type area

For each plot of the hypothetical farm extent, the annual crop type was mapped (not shown here) and validated using Planet imagery, during the period 2018-2021.

In this section, we show how the data cube can help to report on the annual crop type area at farm scale.

#### Select a crop type of interest

Please select a crop type of interest. You can select one specific crop type or consider all of them

In [ ]:
crop_type = crop_type_widget()
crop_type

#### Report

In [ ]:
# Create report
report = report_crop_type_area(field_plots, crop_type.value)

# Print report
for year in report:
    print("\n  In " + year + ":")
    for crop in list(report[year].keys()):
        print(str(round(report[year][crop], 2)) + " ha of " + crop)

## Mapping crop rotation at farm scale

Run the cell below to visualize crop rotation between 2018 and 2021 at farm scale.

In [ ]:
play_crop_rotation(field_plots)

## Mapping crop rotation at plot scale

### Select a field plot of interest

#### Map all the field plots

In [ ]:
# Run display tool
m, option_widget = draw_select(geo_data)
m.add_layer(geo_data)
m

#### Select a field plot of interest

In this section, we will interact with the above display tool to select a field plot from the shapefile. 

Unfortunately, currently, there is no ipython library available to click-select. As part of the Living Wales, we have developed an alternative to allow you to select a polygon feature.

- TO SELECT: use the &#9726; *'draw a rectangle'* symbol in the leftside bar, draw a small polygon **inside** the polygon you would like to select and click on the 'Select' button
- TO DELETE: at any time, you can click on the bin symbol and choose 'Clear All" to delete the polygon(s) you have drawn.


#### Use the selected field plot as the geopolygon of interest

In [ ]:
# Select
my_geopolygon, feature = geom_fromdrawn(option="Selection", shapefile=field_plots)

print("The geopolygon of interest is the following one; ")
my_geopolygon

### Loading data from datacube 

#### Prepare query for datacube

In [ ]:
# Use the whole study period as period of interest
start_date = "2017-10-01"
end_date = "2021-11-01"

# Prepare query
query = {
    "product": "sen1_rtc_pyroSNAP",
    "geopolygon": my_geopolygon,
    "time": (start_date, end_date),
    "output_crs": "epsg:27700",
    "measurements": ["VH", "VV"],
    "dask_chunks": {"y": 2048, "x": 2048},
    "resolution": (-10, 10),
}

#### Load data

In [ ]:
# Preparing query for datacube
start_time = now()
# Let's load the dataset
dataset_in = dc.load(**query)

# define nodata
dataset_in = dataset_in.where(dataset_in != 0)

# clean data: drop dates with only nodata for the area of interest
dataset_in = dataset_in.dropna("time", how="all")

# When using epsg other than 4326 (here: 27700), latitude and longitude are renamed y and x.
# Let's correct that and rename x and y with explicit names
dataset_in = dataset_in.rename({"x": "longitude", "y": "latitude"})

# group images by YYYY-MM-DD
s1_dataset = dataset_in.groupby(dataset_in.time.dt.strftime("%Y-%m-%d")).mean("time")
s1_dataset = s1_dataset.rename({"strftime": "time"})

print("Datacube ready")
print(
    "Took "
    + str(round(now() - start_time, 2))
    + " seconds to request "
    + str(len(s1_dataset.time))
    + " images from datacube."
)

In [ ]:
# Visualise VH backscatter for the first date in the dataset
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

s1_dataset.VH.isel(time=0).plot();

### Masking data out of the field plot

As you can see on the above figure, within the data cube, data are loaded as a geobox (i.e., a rectangle). 
In our case, in cell 9, we have queried a geopolygon. However, the data cube uses the maximum/minimum latitude/longitude of the polygon to load the data.

In this section, we will mask out the pixels which are not included within the draw/selected geopolygon using one of the tools developed by Living Wales.

In [ ]:
# Masking data out of the field plot
s1_dataset = geopolygon_masking(s1_dataset, my_geopolygon)

# Visualise VH backscatter for the first date in the dataset (after masking)
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

s1_dataset.VH.isel(time=0).plot();

### Mapping multi-year rotation

As with Sentinel-2, indices can be calculated using Sentinel-1. The polarization ratio is one of the most used index when analysing radar data.

In this section, for each available date, from the beginning of the period of interest (i.e., Oct 2017) to the end (i.e., Nov 2021), we calculate the parcel backscatter/index and plot its evolution.

In [ ]:
# Calculate VH/VV ratio for each pixel
s1_dataset["Ratio"] = s1_dataset.VH - s1_dataset.VV

# Summarise the polarization ratio at parcel level
median_Ratio = s1_dataset.Ratio.median(["latitude", "longitude"])

In [ ]:
# Plotting the parcel backscatters/indices
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

# define figure size
fig, (ax1) = plt.subplots(1, figsize=(18, 6))
# set title
fig.suptitle("Evolution of the parcel between Oct 2017 and Nov 2021", y=1, fontsize=14)
# set date format for the a axises
dates = [dt.datetime.strptime(d, "%Y-%m-%d").date() for d in median_Ratio.time.values]

# Plot the  ratio
ax1.plot(dates, median_Ratio.values, marker="o")
ax1.grid(visible=True, which="major", axis="both")
ax1.set_ylim([-16, -2])
ax1.xaxis.set_major_locator(mdates.MonthLocator((1, 4, 7, 10)))
ax1.xaxis.set_major_formatter(DateFormatter("%b-%Y"))
ax1.tick_params("x", labelrotation=45)
ax1.set_ylabel("Ratio")

ax1.axvline(x=dt.date(2018, 10, 1), color="lightcoral", linewidth=1.5)
ax1.axvline(x=dt.date(2019, 10, 1), color="lightcoral", linewidth=1.5)
ax1.axvline(x=dt.date(2020, 10, 1), color="lightcoral", linewidth=1.5)

crop = field_plots[field_plots.id == int(feature)]
ax2 = ax1.twiny()
ax2.set_xticks(ticks=[0.15, 0.38, 0.60, 0.85])
ax2.tick_params(axis="x", which="major", labelsize=12)
ax2.set_xticklabels(
    labels=[
        crop["Year_2018"].values[0],
        crop["Year_2019"].values[0],
        crop["Year_2020"].values[0],
        crop["Year_2021"].values[0],
    ],
    color="red",
);